In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('input_last_version_1.csv')

In [ ]:
# filteres the HostName with the count number is less than 16
counts = df['HostName'].value_counts()
filtered_df = df[df['HostName'].isin(counts[counts < 16].index)].reset_index(drop=True)

In [ ]:
decoded_responses_list = filtered_df['HostName'].unique().tolist()

In [ ]:
#connects to abuseipdb and checks if the IP is reported in the last 90 days

import requests
import json

# Define the api-endpoint
url = 'https://api.abuseipdb.com/api/v2/check'

# Create an empty list to store data
data_list = []

for decoded_response in decoded_responses_list:
    querystring = {
        'ipAddress': decoded_response,
        'maxAgeInDays': '90'
    }

    headers = {
        'Accept': 'application/json',
        'Key': 'Use your own key'
    }

    response = requests.get(url=url, headers=headers, params=querystring)

    # Check if the response is successful (status code 200)
    if response.status_code == 200:
        result = response.json()
        ip_address = result['data']['ipAddress']
        last_reported_at = result['data']['lastReportedAt']

        # Append data to the list
        data_list.append({'IPAddress': ip_address, 'LastReportedAt': last_reported_at})
    else:
        print(f"Error for {decoded_response}: {response.status_code}")

# Check if data_list is not empty before creating the DataFrame
if data_list:
    # Convert the list of dictionaries to a DataFrame
    df = pd.DataFrame(data_list)

    # Display the DataFrame
    print(df)
else:
    print("No data to create DataFrame.")

In [ ]:
df.to_csv('IP_last_seen', index=False)